In [ ]:
import json
import os
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import shutil
import xml.etree.ElementTree as ET
from matplotlib.patches import Rectangle
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from tqdm import tqdm

In [ ]:
import os
import xml.etree.ElementTree as ET
import json
from PIL import Image

def convert_xml_to_coco_and_segment(xml_file, output_json, images_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    tree = ET.parse(xml_file)
    root = tree.getroot()

    coco = {
        "info": {},
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

    categories = {}
    category_id = 1
    for label in root.findall('.//label'):
        category_name = label.find('name').text
        if category_name.lower() == 'текст':
            continue
        categories[category_name] = category_id
        coco['categories'].append({
            "id": category_id,
            "name": category_name,
            "supercategory": "none"
        })
        category_id += 1

    image_id = 1
    annotation_id = 1
    min_segment_size = 420
    max_segment_size = 640

    for image in tqdm(root.findall('.//image')):
        file_name = image.get('name')
        image_path = os.path.join(images_folder, file_name)
        if not os.path.exists(image_path):
            print(f"Image file {image_path} does not exist.")
            continue
        img = Image.open(image_path)
        original_width, original_height = img.size

        # Calculate non-intersecting segments
        boxes = []
        for box in image.findall('.//box'):
            label = box.get('label')
            if label not in categories:
                continue
            xmin = float(box.get('xtl'))
            ymin = float(box.get('ytl'))
            xmax = float(box.get('xbr'))
            ymax = float(box.get('ybr'))
            boxes.append((xmin, ymin, xmax, ymax, categories[label]))

        # Create segments for each box
        for box in boxes:
            box_xmin, box_ymin, box_xmax, box_ymax, category_id = box

            # Calculate the segment coordinates
            start_x = max(0, box_xmax - max_segment_size)
            end_x = min(box_xmin, original_width - min_segment_size)
            start_y = max(0, box_ymax - max_segment_size)
            end_y = min(box_ymin, original_height - min_segment_size)

            end_x = min(start_x + max_segment_size, original_width)
            end_y = min(start_y + max_segment_size, original_height)

            if end_x - start_x < min_segment_size:
                start_x = max(0, end_x - min_segment_size)
            if end_y - start_y < min_segment_size:
                start_y = max(0, end_y - min_segment_size)

            segment = img.crop((start_x, start_y, end_x, end_y))
            segment_file_name = f"segment_{image_id}_{start_x}_{start_y}.png"
            segment_path = os.path.join(output_folder, segment_file_name)
            segment.save(segment_path)

            # Update COCO image data
            coco['images'].append({
                "id": image_id,
                "file_name": segment_file_name,
                "height": end_y - start_y,
                "width": end_x - start_x
            })

            # Update annotations for this segment
            for box in boxes:
                if start_x < box[2] and end_x > box[0] and start_y < box[3] and end_y > box[1]:
                    xmin = max(box[0] - start_x, 0)
                    ymin = max(box[1] - start_y, 0)
                    xmax = min(box[2] - start_x, end_x - start_x)
                    ymax = min(box[3] - start_y, end_y - start_y)
                    width = xmax - xmin
                    height = ymax - ymin


                    coco['annotations'].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": box[4],
                        "bbox": [xmin, ymin, width, height],
                        "area": width * height,
                        "segmentation": [],
                        "iscrowd": 0
                    })
                    annotation_id += 1

            image_id += 1

    # Save the updated COCO data
    with open(output_json, 'w') as json_file:
        json.dump(coco, json_file, indent=4)

    print(f"Conversion completed. {len(coco['images'])} images and {len(coco['annotations'])} annotations created.")

# Example usage:
# convert_xml_to_coco_and_segment('path_to_xml_file.xml', 'output.json', 'images_folder', 'output_folder')


In [ ]:
for i in [330, 482, 331, 333, 334]:
    cvat_json_path = f'/kaggle/input/electrocv/EKF AI Challenge/{i}/annotations.xml'
    output_coco_json_path = f'/kaggle/working/coco_{i}.json'
    images_folder = f'/kaggle/input/electrocv/EKF AI Challenge/{i}/images'
    convert_xml_to_coco_and_segment(cvat_json_path, output_coco_json_path, images_folder, f'/kaggle/working/{i}/')

In [ ]:
import os
import json
from shutil import copy2

def merge_datasets(image_folders, json_files, output_image_folder, output_json_file):
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)

    merged_coco = {
        "info": {},
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }

    # Обработка аннотаций и копирование изображений
    image_id = 1
    annotation_id = 1
    category_ids = {}
    category_id = 1
    category_id_mapping = {}  # Новый словарь для сопоставления старых и новых ID категорий
    
    for json_file in json_files:
        with open(json_file, 'r') as file:
            coco_data = json.load(file)
            
            # Обработка категорий
            if 'categories' in coco_data:
                for category in coco_data['categories']:
                    if category['name'] not in category_ids:
                        category_ids[category['name']] = category_id
                        category_id_mapping[category['id']] = category_id
                        new_category = category.copy()
                        new_category['id'] = category_id
                        merged_coco['categories'].append(new_category)
                        category_id += 1
                    else:
                        category_id_mapping[category['id']] = category_ids[category['name']]
            
            # Обработка изображений и аннотаций
            if 'images' in coco_data:
                for image in coco_data['images']:
                    new_image = image.copy()
                    new_image['id'] = image_id
                    # Копирование изображений в новую папку
                    source_path = os.path.join(image_folders[json_files.index(json_file)], image['file_name'])
                    destination_file_name = f"{image_id}_{os.path.basename(image['file_name'])}"  # Новый уникальный имя файла
                    destination_path = os.path.join(output_image_folder, destination_file_name)
                    copy2(source_path, destination_path)
                    new_image['file_name'] = destination_file_name
                    merged_coco['images'].append(new_image)
                    
                    # Обновление аннотаций
                    for annotation in coco_data['annotations']:
                        if annotation['image_id'] == image['id']:
                            new_annotation = annotation.copy()
                            new_annotation['id'] = annotation_id
                            new_annotation['image_id'] = image_id
                            new_annotation['category_id'] = category_id_mapping[annotation['category_id']]
                            merged_coco['annotations'].append(new_annotation)
                            annotation_id += 1
                    
                    image_id += 1

    # Сохранение объединённого файла JSON
    with open(output_json_file, 'w') as file:
        json.dump(merged_coco, file, indent=4)

# Пути к папкам с изображениями и файлами JSON
image_folders = ["/kaggle/working/331", "/kaggle/working/330", "/kaggle/working/334", "/kaggle/working/482", "/kaggle/working/333"]
json_files = ["/kaggle/working/coco_331.json", "/kaggle/working/coco_330.json", "/kaggle/working/coco_334.json", "/kaggle/working/coco_482.json", "/kaggle/working/coco_333.json"]

# Вызов функции для объединения
merge_datasets(image_folders, json_files, "merged_dataset1", "merged_annotations1.json")


In [ ]:


# Define the augmentation pipeline
def get_rotation_transforms(angle):
    return A.Compose([
        A.Rotate(limit=(angle, angle), p=1.0),  # Rotate by a specific angle with 100% probability
        A.RandomScale(scale_limit=0.1, p=0.1),  # Scale with low probability
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.1),  # Add noise with low probability
        ToTensorV2()
    ], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

# Load COCO annotations
def load_coco_annotations(coco_json_path):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    return coco_data

# Save augmented COCO annotations
def save_coco_annotations(coco_data, output_json_path):
    with open(output_json_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

# Augment images and annotations
def augment_images_and_annotations(coco_data, images_folder, output_images_folder):
    rotation_angles = [90, 180, 270]
    augmented_images = []
    augmented_annotations = []
    image_id = 1
    annotation_id = 1

    for image_info in tqdm(coco_data['images']):
        image_path = os.path.join(images_folder, image_info['file_name'])
        image = cv2.imread(image_path)
        if image is None:
            continue

        annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == image_info['id']]
        bboxes = [ann['bbox'] for ann in annotations]
        category_ids = [ann['category_id'] for ann in annotations]

        # Save original image and annotations
        original_image_filename = f'original_image_{image_id}.jpg'
        original_image_path = os.path.join(output_images_folder, original_image_filename)
        cv2.imwrite(original_image_path, image)

        augmented_images.append({
            "id": image_id,
            "file_name": original_image_filename,
            "height": image.shape[0],
            "width": image.shape[1]
        })

        for bbox, category_id in zip(bboxes, category_ids):
            augmented_annotations.append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": bbox,
                "area": bbox[2] * bbox[3],
                "segmentation": [],
                "iscrowd": 0
            })
            annotation_id += 1

        # Save augmented images and annotations for each rotation
        for angle in rotation_angles:
            transforms = get_rotation_transforms(angle)
            augmented = transforms(image=image, bboxes=bboxes, category_ids=category_ids)
            augmented_image = augmented['image']
            augmented_bboxes = augmented['bboxes']
            augmented_category_ids = augmented['category_ids']

            augmented_image_filename = f'augmented_image_{image_id}_{angle}.jpg'
            augmented_image_path = os.path.join(output_images_folder, augmented_image_filename)
            cv2.imwrite(augmented_image_path, augmented_image.permute(1, 2, 0).numpy())

            augmented_images.append({
                "id": image_id + angle // 90,  # Ensure unique IDs for augmented images
                "file_name": augmented_image_filename,
                "height": augmented_image.shape[1],
                "width": augmented_image.shape[2]
            })

            for bbox, category_id in zip(augmented_bboxes, augmented_category_ids):
                augmented_annotations.append({
                    "id": annotation_id,
                    "image_id": image_id + angle // 90,
                    "category_id": category_id,
                    "bbox": bbox,
                    "area": bbox[2] * bbox[3],
                    "segmentation": [],
                    "iscrowd": 0
                })
                annotation_id += 1

        image_id += 4  # Increment by 4 to account for the original and three augmented images

    coco_data['images'] = augmented_images
    coco_data['annotations'] = augmented_annotations
    return coco_data

In [ ]:
coco_json_path = f'/kaggle/working/merged_annotations1.json'
images_folder = f'/kaggle/working/merged_dataset1/'
output_images_folder = f'/kaggle/working/merged_dataset2'

# Load annotations
coco_data = load_coco_annotations(coco_json_path)

# Create output folder if it doesn't exist
if not os.path.exists(output_images_folder):
    os.makedirs(output_images_folder)

# Augment images and annotations
coco=augment_images_and_annotations(coco_data, images_folder, output_images_folder)
save_coco_annotations(coco, f'aug_merged_annotations1.json')

In [ ]:
len(os.listdir('/kaggle/working/merged_dataset2'))

In [ ]:
!pip install ultralytics

In [ ]:
from tqdm import tqdm
import os
import json
from pathlib import Path
import yaml
from random import shuffle
def convert_coco_to_yolo(coco_json_path, output_dir):
    # Создаем директории для изображений
    train_img_dir = Path(output_dir) / "images" / "train"
    val_img_dir = Path(output_dir) / "images" / "val"
    train_img_dir.mkdir(parents=True, exist_ok=True)
    val_img_dir.mkdir(parents=True, exist_ok=True)
    
    # Создаем директории для меток
    train_lbl_dir = Path(output_dir) / "labels" / "train"
    val_lbl_dir = Path(output_dir) / "labels" / "val"
    train_lbl_dir.mkdir(parents=True, exist_ok=True)
    val_lbl_dir.mkdir(parents=True, exist_ok=True)
    
    # Загружаем аннотации COCO
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # Извлекаем категории
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
    
    # Разделяем данные на обучающую, валидационную и тестовую выборки
    img_files = [img['file_name'] for img in coco_data['images']]
    shuffle(img_files)
    train_files = img_files[:18000]
    val_files = img_files[18000:]
    
    # Создаем словарь для быстрого доступа к изображениям по имени файла и к аннотациям по image_id
    img_dict = {img['file_name']: img for img in coco_data['images']}
    ann_dict = {}
    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        if image_id not in ann_dict:
            ann_dict[image_id] = []
        ann_dict[image_id].append(ann)
    
    def save_files(files, img_target_dir, lbl_target_dir):
        for img_file in tqdm(files):
            img_path = img_target_dir / img_file
            
            # Копируем изображение в соответствующую директорию
            original_img_path = Path(coco_json_path).parent / 'merged_dataset2' / img_file
            img_path.write_bytes(original_img_path.read_bytes())
            
            # Создаем соответствующий файл аннотации
            label_file = lbl_target_dir / f"{img_path.stem}.txt"
            with open(label_file, 'w') as label_f:
                img_info = img_dict[img_file]
                image_id = img_info['id']
                
                if image_id in ann_dict:
                    for ann in ann_dict[image_id]:
                        category_id = ann['category_id'] - 1  # YOLO использует 0-индексацию категорий
                        bbox = ann['bbox']
                        # Конвертируем bbox в формат YOLO
                        x_center = (bbox[0] + bbox[2] / 2) / img_info['width']
                        y_center = (bbox[1] + bbox[3] / 2) / img_info['height']
                        width = bbox[2] / img_info['width']
                        height = bbox[3] / img_info['height']
                        
                        label_f.write(f"{category_id} {x_center} {y_center} {width} {height}\n")
    
    # Сохраняем файлы
    save_files(train_files, train_img_dir, train_lbl_dir)
    save_files(val_files, val_img_dir, val_lbl_dir)
    
    # Создаем файл train.yaml
    train_yaml_content = {
        'path': '/kaggle/working/datasets',
        'train': str(train_img_dir.resolve()),
        'val': str(val_img_dir.resolve()),
        'nc': len(categories),
        'names': list(categories.values())
    }
    train_yaml_path = Path(output_dir) / "train.yaml"
    with open(train_yaml_path, 'w') as f:
        yaml.dump(train_yaml_content, f)

# Пример использования
coco_json_path = '/kaggle/working/aug_merged_annotations1.json'
output_dir = '/kaggle/working/datasets'
convert_coco_to_yolo(coco_json_path, output_dir)


In [ ]:
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
from ultralytics import YOLO

# Предполагается, что 'last.pt' - это файл весов последней эпохи, который вы хотите загрузить.
# Укажите корректный путь к файлу последней сохранённой модели.
model = YOLO('/kaggle/input/electrocv/epoch8.pt')  # Замените на актуальный путь к вашему файлу с весами

# Продолжить обучение модели с места последнего сохранения
model.train(
    data='/kaggle/working/datasets/train.yaml',
    epochs=100,  # Общее количество эпох, до которого вы хотите обучить модель
    resume=True,  # Указывает модели продолжить с последнего момента сохранения
    save_period=1  # Период сохранения модели
)